In [1]:
# Importando bibliotecas
import re
#import dask
import warnings
import numpy as np
import pandas as pd
import pyomo.environ as pyo
#import modin.pandas as pdm
import pyarrow.parquet as pq
import dask.dataframe as dd

from rapidfuzz import process

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option("display.max_colwidth", None)

#pd.reset_option("display.max_rows")

# Verifica o uso de CPU
#dask.system.cpu_count()

# Dados

Os dados utilizados na pesquisa e resultado de informações, de origens publica e restrita tendo como fontes o portal do [**Justiça e Números - Indicadores**](https://justica-em-numeros.cnj.jus.br/painel-estatisticas/), [**Portal da Tranparência do TJBA**](https://www.tjba.jus.br/transparencia/), [**DataJude**](https://www.cnj.jus.br/sistemas/datajud/) e PJe em parceria com o TJBA.

Os dados no Justiça e Números foram extraidos em 11/08/2025 com informações do: Painel atualizado em 25/07/2025, com dados recebidos dos tribunais e processados até 16/07/2025 (situações processuais até 30/06/2025)

Diretódio do projeto onde estão os dados:

```
dados
├── DADOS_TJBA_PG.parquet - Arquivo gerado pelo Apache Hop no cruzaemnto de dados do 1º Grau
├── MEMBROS-DO-TJBA-E-AGENTES-PUBLICOS.csv - Arquivo do PT com lista de magistrados e lotação
└── CNJ_TJBA_JULGADOS.csv - Arquivo do JN com processos julgados TJBA já filtrados do 1º/TJBA
```

## Dados de Magistrados - Portal da Transparência
Esses dados podem demosntrar quantos magistrados exitem por vara.

In [39]:
# Inportando dados de magistrados do PT
df_magistrados = pd.read_csv('../../dados/MEMBROS_E_AGENTES_PUBLICOS_JUL2025.csv', sep=';')
df_magistrados = df_magistrados[df_magistrados['Cargo'].str.startswith('JUIZ', na=False)]

df_magistrados.head()

,Nome,Matrícula Funcional,Cargo,Função de Confiança/ Cargo em Comissão,Lotação,Ato de Provimento,Data de Publicação
7,ABRAAO BARRETO CORDEIRO,9679782,JUIZ DE DIREITO,NaN,VARA CRIM JURI EXEC PENAIS MEN,PORT S/N,16/09/2013
25,ADALBERTO LIMA BORGES FILHO,9698345,JUIZ DE DIREITO,NaN,JURISDICAO PLENA,PORT S/N,06/10/2021
69,ADERALDO DE MORAIS LEITE JUNIOR,9679871,JUIZ DE DIREITO,NaN,JURISDICAO PLENA,PORT S/N,16/09/2013
73,ADIANE JAQUELINE NEVES DA SILVA OLIVEIRA,9679812,JUIZ DE DIREITO,NaN,1ª VARA DOS SISTEMA DOS JUIZADOS ESPECIAIS,PORT S/N,16/09/2013
74,ADIDA ALVES DOS SANTOS,8098654,JUIZ DE DIREITO,NaN,2ª VARA DA INFANCIA E JUVENTUDE,PORT S/N,01/12/2004


In [40]:
df_magistrados.shape

(621, 7)

In [41]:
df_magistrados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 621 entries, 7 to 8846
Data columns (total 7 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Nome                                    621 non-null    object
 1   Matrícula Funcional                     621 non-null    int64 
 2   Cargo                                   621 non-null    object
 3   Função de Confiança/ Cargo em Comissão  0 non-null      object
 4   Lotação                                 621 non-null    object
 5   Ato de Provimento                       621 non-null    object
 6   Data de Publicação                      621 non-null    object
dtypes: int64(1), object(6)
memory usage: 38.8+ KB


In [42]:
# Contagens de Juízes
df_magistrados.Cargo.value_counts()

Cargo
JUIZ DE DIREITO                                      544
JUIZ SUBSTITUTO                                       56
JUIZ SUBSTITUTO DE SEGUNDO GRAU - ENTRANCIA FINAL     21
Name: count, dtype: int64

In [43]:
# Pesquisa por magistrado
df_magistrados[df_magistrados['Nome'] == 'ULYSSES MAYNARD SALGADO']

,Nome,Matrícula Funcional,Cargo,Função de Confiança/ Cargo em Comissão,Lotação,Ato de Provimento,Data de Publicação
8430,ULYSSES MAYNARD SALGADO,8088900,JUIZ DE DIREITO,NaN,1A VARA FAZENDA PUBLICA,PORT S/N,23/12/2002


In [50]:
# Contagem de Lotação
df_magistrados_g = df_magistrados.groupby('Lotação').agg(quant=('Lotação', 'count'))

In [53]:
df_magistrados_g

,quant
Lotação,
1ª VARA CRIM JURI EXEC PENAIS,1
VARA EXEC. PENAIS E MEDIDAS ALTERNATIVAS,1
10ª VARA CIVEL,1
10ª VARA DE FAMILIA,1
10ª VARA DE RELACOES DE CONSUMO,1
...,...
VARA ESP CRIM INFAN JUVEN,1
VARA FEITOS RELATIVOS REL DE CONSUMO CIVEIS E COM,1
VARA FEITOS RELATIVOS REL DE CONSUMO CIVEIS E COM,1


In [55]:
# Vara com maior número de juízes atuantes
df_magistrados_g[df_magistrados_g['quant'] == df_magistrados_g['quant'].max()]

,quant
Lotação,
JURISDICAO PLENA,88


## Dados de Processos Julgados - Indicadores CNJ
Esses dados foram baixados do **Justiça em Números** mantido pelo CNJ, tendo como orijem dos dados o **DataJud**. Nesse conjunto de dados temos processos que foram julgados nos anos de 2024 e 2025.

In [12]:
# Importando dados do CNJ, processos julgados
df_cnj = pd.read_csv('../../dados/CNJ_TJBA_JULGADOS.csv', encoding='utf8', sep=';')
df_cnj.head()

,Tribunal,Grau,Nome_Orgao,UF,Municipio,Ano,Mes,Processo,Codigos_classes,Codigo_da_Ultima_classe,Nome_da_Ultima_classe,Codigos_assuntos,Data_de_Referencia,Data_do_julgamento:_Situacao:_Movimento,Data_da_decisao:_Situacao:_Movimento,Formato,Procedimento,Recurso,Codigo_Orgao,id_municipio,Polo_ativo,Polo_ativo_-_CNPJ,Polo_ativo_-_Natureza_juridica,Polo_ativo_-_CNAE,Polo_passivo,Polo_passivo_-_CNPJ,Polo_passivo_-_Natureza_juridica,Polo_passivo_-_CNAE,Poder_publico
0,TJBA,G1,VARA DE JURISDIÇÃO PLENA - ITAPICURU,BA,ITAPICURU,2024,3,8000013-59.2017.8.05.0127,{1116},1116,EXECUÇÃO FISCAL,{5952},2024-03-26,{20240326:28:459},{},Eletrônico,Execução fiscal,0,5312,2349,{MUNICIPIO DE ITAPICURU},{13647557000160},{MUNICIPIO},{ADMINISTRACAO PUBLICA EM GERAL},NaN,NaN,NaN,NaN,Poder_Publico-Polo_Ativo
1,TJBA,G1,VARA DE JURISDIÇÃO PLENA - ITAPICURU,BA,ITAPICURU,2024,4,8000014-44.2017.8.05.0127,{1116},1116,EXECUÇÃO FISCAL,{5952},2024-04-10,{20240410:28:461},{},Eletrônico,Execução fiscal,0,5312,2349,{MUNICIPIO DE ITAPICURU},{13647557000160},{MUNICIPIO},{ADMINISTRACAO PUBLICA EM GERAL},NaN,NaN,NaN,NaN,Poder_Publico-Polo_Ativo
2,TJBA,G1,VARA DE JURISDIÇÃO PLENA - ITAPICURU,BA,ITAPICURU,2024,3,8000894-36.2017.8.05.0127,{1116},1116,EXECUÇÃO FISCAL,{5952},2024-03-26,{20240326:28:459},{},Eletrônico,Execução fiscal,0,5312,2349,{MUNICIPIO DE ITAPICURU},{13647557000160},{MUNICIPIO},{ADMINISTRACAO PUBLICA EM GERAL},NaN,NaN,NaN,NaN,Poder_Publico-Polo_Ativo
3,TJBA,G1,VARA DE JURISDIÇÃO PLENA - ITAPICURU,BA,ITAPICURU,2024,3,8000012-74.2017.8.05.0127,{1116},1116,EXECUÇÃO FISCAL,{5952},2024-03-26,{20240326:28:459},{},Eletrônico,Execução fiscal,0,5312,2349,{MUNICIPIO DE ITAPICURU},{13647557000160},{MUNICIPIO},{ADMINISTRACAO PUBLICA EM GERAL},NaN,NaN,NaN,NaN,Poder_Publico-Polo_Ativo
4,TJBA,G1,VARA DE JURISDIÇÃO PLENA - ITAPICURU,BA,ITAPICURU,2024,3,0000753-32.2012.8.05.0127,{1116},1116,EXECUÇÃO FISCAL,{10887},2024-03-26,{20240326:28:459},{},Eletrônico,Execução fiscal,0,5312,2349,{INSTITUTO NACIONAL DE METROLOGIA QUALIDADE E TECNOLOGIA - INMETRO.},{00662270000168},{AUTARQUIA FEDERAL},{ADMINISTRACAO PUBLICA EM GERAL},{G D DE SOUZA DE ITAPICURU},{07593292000162},{EMPRESARIO (INDIVIDUAL)},{FABRICACAO DE PRODUTOS DE PADARIA E CONFEITARIA COM PREDOMINANCIA DE PRODUCAO PROPRIA},Poder_Publico-Polo_Ativo


In [7]:
df_cnj.shape

(1440265, 29)

In [ ]:
# Verificando anos no df do CNJ
df_cnj['Ano'].unique()

array([2024, 2025])

In [13]:
# Retrigindo somente ao ano de 2024
df_cnj = df_cnj[df_cnj['Ano'] == 2024]
df_cnj.shape

(1058211, 29)

In [14]:
# Processos que tiveream mais de um julgamento
df_cnj['Processo'].value_counts()[lambda x: x > 1]

Processo
8005064-54.2021.8.05.0113    7
0700281-17.2021.8.05.0113    7
0001917-26.2002.8.05.0113    6
8005315-83.2022.8.05.0001    6
8000342-96.2024.8.05.0201    5
                            ..
0804903-37.2017.8.05.0001    2
0774111-71.2015.8.05.0001    2
8001014-66.2024.8.05.0149    2
8030198-02.2019.8.05.0001    2
8060266-61.2021.8.05.0001    2
Name: count, Length: 26470, dtype: int64

## Cruzamento dos Dados

Nessa fase precisamos cruzar alguns dados, apesar de termos fontes como **CNJ(DataJUD)**, algumas informações são mais específicas e precisamos cruzar essas informações com o TJBA. Nessa primeira etapa devido ao volume e acomplexidade, foi utilizado a ferramenta de ETL [**Apache Hop**](https://hop.apache.org/), para o cruzamento, análises e um tratamento inicial. O artefado final gerado nessa etapa será um arquivo parquet, que usaremos como principal fonte de dados para o modelo.

<p align="center">
    <img width="850" height="250" src="../../img/001.png">
</p>

## Tratamento dos Dados

Aqui vamos trabalhar com os dados gerado na etapa de cruzemnto, realizando alguns tratamentos e adequando os dados;

*	Selecionar as melhores features (colunas).
*	Remover features (colunas) dos df's.
*	Verificar valores ausentes que possam impactar. 
*	Verificar qual fonte dos dados relacionado a sistemas vamos utilizar.
*	Trabalhar na questão dos magistrados.
*	Etc...






### Tratando Dados do Portal da Transparência

In [ ]:
# Alguns nomes do df de cadastro (PT) não batem com o do TJBA, ajustando o cadastro por similaridade
df_magistrados['vm_nome'] = np.nan
nomes_mov = df_tjba['vm_magistrado'].dropna().unique()

for i, row in df_magistrados.iterrows():
    nome_cadastro = row['Nome']
    match, score, idx = process.extractOne(nome_cadastro, nomes_mov)
    
    if score > 85:  # limiar de confiança
        df_magistrados.at[i, 'vm_nome'] = match

In [11]:
df_magistrados.head()

,Nome,Matrícula Funcional,Cargo,Função de Confiança/ Cargo em Comissão,Lotação,Ato de Provimento,Data de Publicação,vm_nome
7,ABRAAO BARRETO CORDEIRO,9679782,JUIZ DE DIREITO,NaN,VARA CRIM JURI EXEC PENAIS MEN,PORT S/N,16/09/2013,ABRAAO BARRETO CORDEIRO
25,ADALBERTO LIMA BORGES FILHO,9698345,JUIZ DE DIREITO,NaN,JURISDICAO PLENA,PORT S/N,06/10/2021,ADALBERTO LIMA BORGES FILHO
69,ADERALDO DE MORAIS LEITE JUNIOR,9679871,JUIZ DE DIREITO,NaN,JURISDICAO PLENA,PORT S/N,16/09/2013,ADERALDO DE MORAIS LEITE JUNIOR
73,ADIANE JAQUELINE NEVES DA SILVA OLIVEIRA,9679812,JUIZ DE DIREITO,NaN,1ª VARA DOS SISTEMA DOS JUIZADOS ESPECIAIS,PORT S/N,16/09/2013,ADIANE JAQUELINE NEVES DA SILVA
74,ADIDA ALVES DOS SANTOS,8098654,JUIZ DE DIREITO,NaN,2ª VARA DA INFANCIA E JUVENTUDE,PORT S/N,01/12/2004,ADIDA ALVES DOS SANTOS


In [12]:
# Verificando se todos os nomes foram preenchidos
df_magistrados[df_magistrados['vm_nome'].isna()]

,Nome,Matrícula Funcional,Cargo,Função de Confiança/ Cargo em Comissão,Lotação,Ato de Provimento,Data de Publicação,vm_nome


### Tratando Dados do TJBA

In [15]:
# Verificando eschema do arquivo parquet gerado
table = pq.read_table("../../dados/DADOS_TJBA_PG.parquet")
print(table.schema)

pje_processo: string
vm_processo: string
vm_dt_recebimento: timestamp[ms, tz=UTC]
vm_dt_movimento: timestamp[ms, tz=UTC]
vm_cod_cnj_comarca: int64
vm_mapa_comarca: int64
vm_cod_comarca: int64
vm_comarca: string
pje_id_jurisdicao: int64
pje_ds_jurisdicao: string
vm_mapa_vara: int64
vm_cod_vara: int64
vm_cod_cnj_vara: int64
pje_id_orgao_julgador: int64
pje_nr_vara: int64
vm_vara: string
pje_ds_orgao_julgador: string
pje_cd_classe_judicial: string
vm_cod_cls_cnj: int64
vm_classe: string
pje_ds_classe_judicial: string
pje_ds_natureza: string
pje_vl_peso: int64
pje_cd_assunto_principal: string
pje_ds_assunto_principal: string
vm_cod_mov_cnj: int64
vm_movimento: string
pje_id_competencia: int64
pje_ds_competencia: string
vm_competencia: string
vm_localizacao: string
vm_tp_proc: string
pje_id_orgao_julgador_cargo: int64
pje_ds_orgao_julgador_cargo: string
vm_cod_cnj_magistrado: int64
vm_mapa_magistrado: int64
vm_cod_magistrado: string
vm_magistrado: string
vm_consulta: string
vm_situacao: str

In [16]:
# Importando arquivo parquet gerado pelo Apache Hop no Pandas
df_tjba = pd.read_parquet('../../dados/DADOS_TJBA_PG.parquet', engine="pyarrow", use_pandas_metadata=False)

# Importando arquivo parquet gerado pelo Apache Hop no Dask
#df_tjba = dd.read_parquet('../../dados/DADOS_TJBA_PG.parquet', engine="pyarrow", use_pandas_metadata=False)

# Conver timestamp para datetime
#df["cnj_dt_referencia"] = df["cnj_dt_referencia"].dt.tz_convert(None)
#df["vm_dt_recebimento"] = df["vm_dt_recebimento"].dt.tz_convert(None)
#df["vm_dt_movimento"] = df["vm_dt_movimento"].dt.tz_convert(None)

In [18]:
df_tjba.shape

(11715246, 50)

In [19]:
df_tjba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11715246 entries, 0 to 11715245
Data columns (total 50 columns):
 #   Column                       Dtype              
---  ------                       -----              
 0   pje_processo                 object             
 1   vm_processo                  object             
 2   vm_dt_recebimento            datetime64[ms, UTC]
 3   vm_dt_movimento              datetime64[ms, UTC]
 4   vm_cod_cnj_comarca           int64              
 5   vm_mapa_comarca              int64              
 6   vm_cod_comarca               int64              
 7   vm_comarca                   object             
 8   pje_id_jurisdicao            float64            
 9   pje_ds_jurisdicao            object             
 10  vm_mapa_vara                 int64              
 11  vm_cod_vara                  int64              
 12  vm_cod_cnj_vara              int64              
 13  pje_id_orgao_julgador        float64            
 14  pje_nr_vara     

In [20]:
df_tjba.head()

,pje_processo,vm_processo,vm_dt_recebimento,vm_dt_movimento,vm_cod_cnj_comarca,vm_mapa_comarca,vm_cod_comarca,vm_comarca,pje_id_jurisdicao,pje_ds_jurisdicao,vm_mapa_vara,vm_cod_vara,vm_cod_cnj_vara,pje_id_orgao_julgador,pje_nr_vara,vm_vara,pje_ds_orgao_julgador,pje_cd_classe_judicial,vm_cod_cls_cnj,vm_classe,pje_ds_classe_judicial,pje_ds_natureza,pje_vl_peso,pje_cd_assunto_principal,pje_ds_assunto_principal,vm_cod_mov_cnj,vm_movimento,pje_id_competencia,pje_ds_competencia,vm_competencia,vm_localizacao,vm_tp_proc,pje_id_orgao_julgador_cargo,pje_ds_orgao_julgador_cargo,vm_cod_cnj_magistrado,vm_mapa_magistrado,vm_cod_magistrado,vm_magistrado,vm_consulta,vm_situacao,vm_entrancia,vm_compl_mov,vm_tempo_tramitacao,pje_vl_peso_processual,vm_prioridade,vm_ds_prioridade,pje_vl_peso_prioridade,vm_concluso,vm_sistema,vm_num_ativo
0,8003619-75.2022.8.05.0271,8003619-75.2022.8.05.0271,2022-09-19 14:02:01+00:00,2024-08-26 12:47:34+00:00,5359,403,1255,VALENCA,255.0,VALENÇA,1642,11534,50076,534.0,1248.0,2ª VARA DE FEITOS DE REL DE CONS. CÍVEL E COMERCIAIS,2ª V DOS FEITOS DE REL. DE CONS. CÍVEIS COM. FAZ. PUB. E ACID. TRAB. DE VALENÇA,65,65,ACAO CIVIL PUBLICA,AÇÃO CIVIL PÚBLICA,CONHECIMENTO,2.0,10113,Flora,11010,PROFERIDO DESPACHO DE MERO EXPEDIENTE,9.0,Fazenda Pública - Atos Administrativos,FAZENDA PUBLICA ATOS ADMINISTRATIVOS,DIREÇÃO DE SECRETARIA,DIGITAL,1125.0,Juiz de Direito Titular,136647.0,378.0,1153579,LEONARDO RULIAN CUSTODIO,DESPACHO,EM ANDAMENTO,FINAL,,0.0,4.0,N,None,0.0,0,PJEPG,1
1,8003068-15.2022.8.05.0039,8003068-15.2022.8.05.0039,2022-02-06 18:58:24+00:00,2024-09-19 15:03:05+00:00,903,68,115,CAMACARI,15.0,CAMAÇARI,2004,11453,79440,453.0,45.0,2ª VARA DA FAZENDA PÚBLICA,2ª V DE FAZENDA PUBLICA DE CAMAÇARI,1116,1116,EXECUCAO FISCAL,EXECUÇÃO FISCAL,EXECUçãO,2.0,6017,Dívida Ativa (Execução Fiscal),461,EXTINTO O PROCESSO POR AUSENCIA DAS CONDICOES DA ACAO,6.0,Fazenda Pública - Tributos,FAZENDA PUBLICA TRIBUTOS,DIREÇÃO DE SECRETARIA,DIGITAL,935.0,JUIZ DE DIREITO,4690.0,166.0,1858441,DANIEL LIMA FALCAO,JULGAMENTO,BAIXADO,FINAL,SEM MERITO,0.0,4.0,N,None,0.0,0,PJEPG,1
2,0088391-11.2003.8.05.0001,0088391-11.2003.8.05.0001,2003-07-18 13:10:09+00:00,2025-07-18 22:24:10+00:00,4278,334,18,SALVADOR,8.0,SALVADOR - REGIÃO METROPOLITANA,4202,111452,90461,1452.0,15.0,15ª VARA DA FAZENDA PÚBLICA,15ª V DA FAZENDA PÚBLICA DE SALVADOR,1116,1116,EXECUCAO FISCAL,EXECUÇÃO FISCAL,EXECUçãO,2.0,12989,Execução Fiscal,11010,PROFERIDO DESPACHO DE MERO EXPEDIENTE,9.0,Fazenda Pública - Atos Administrativos,FAZENDA PUBLICA ATOS ADMINISTRATIVOS,DIREÇÃO DE SECRETARIA,DIGITAL,2853.0,Juiz de Direito Titular,4720.0,161.0,12898787,CRISTIANE MENEZES SANTOS BARRETO,DESPACHO,EM ANDAMENTO,FINAL,,0.0,4.0,N,None,0.0,0,PJEPG,1
3,0000087-73.2019.8.05.0066,0000087-73.2019.8.05.0066,2019-03-18 18:20:00+00:00,2025-07-09 17:53:34+00:00,1337,103,1165,CONDEUBA,165.0,CONDEÚBA,273,11754,5130,754.0,314.0,VARA JURISDIÇÃO PLENA,VARA CRIMINAL DE CONDEÚBA,283,283,ACAO PENAL PROCEDIMENTO ORDINARIO,AÇÃO PENAL - PROCEDIMENTO ORDINÁRIO,CONHECIMENTO,2.0,11417,Estupro de vulnerável,11010,PROFERIDO DESPACHO DE MERO EXPEDIENTE,53.0,CRIME,CRIME,DIREÇÃO DE SECRETARIA,DIGITAL,1740.0,JUIZ SUBSTITUTO,150543.0,1492.0,13523090,CARLOS TIAGO SILVA ADAES,DESPACHO,EM ANDAMENTO,INICIAL,,0.0,4.0,N,None,0.0,0,PJEPG,1
4,8055437-37.2021.8.05.0001,8055437-37.2021.8.05.0001,2021-05-28 13:47:59+00:00,2025-05-30 17:27:43+00:00,4278,334,18,SALVADOR,8.0,SALVADOR - REGIÃO METROPOLITANA,716,11553,17167,553.0,923.0,6ª VARA CÍVEL E COMERCIAL,6ª V CÍVEL E COMERCIAL DE SALVADOR,7,7,PROCEDIMENTO COMUM CIVEL,PROCEDIMENTO COMUM CÍVEL,CONHECIMENTO,2.0,9597,Seguro,11010,PROFERIDO DESPACHO DE MERO EXPEDIENTE,12.0,Cível,CIVEL,DIREÇÃO DE SECRETARIA,DIGITAL,1165.0,Juiz de Direito Titular,0.0,750.0,1133966,CARLOS CARVALHO RAMOS DE CERQUEIRA JUNIOR,DESPACHO,EM ANDAMENTO,FINAL,,0.0,4.0,N,None,0.0,0,PJEPG,1


In [21]:
# Removendo fetures
df_tjba.drop([
    'pje_processo', 'vm_cod_cnj_comarca', 'vm_mapa_comarca', 'pje_id_jurisdicao', 
    'pje_ds_jurisdicao', 'vm_mapa_vara', 'vm_cod_cnj_vara', 'pje_id_orgao_julgador', 'pje_nr_vara', 
    'pje_ds_orgao_julgador', 'pje_cd_classe_judicial', 'pje_ds_classe_judicial', 'pje_vl_peso', 
    'pje_cd_assunto_principal', 'pje_ds_assunto_principal', 'vm_cod_mov_cnj', 
    'pje_ds_competencia', 'vm_localizacao', 'vm_tp_proc', 'pje_id_orgao_julgador_cargo', 
    'pje_ds_orgao_julgador_cargo', 'vm_cod_cnj_magistrado', 'vm_cod_magistrado', 'vm_entrancia', 'vm_compl_mov',  
    'vm_tempo_tramitacao', 'vm_prioridade', 'vm_ds_prioridade','pje_vl_peso_prioridade', 'vm_concluso', 'vm_num_ativo'
], axis=1, inplace=True) #'vm_dt_recebimento', 'vm_movimento',

In [22]:
# Colocando dados em letras maiúsculas
#df_tjba['pje_ds_assunto_principal'] = df_tjba['pje_ds_assunto_principal'].str.upper()
#df_tjba['pje_ds_orgao_julgador_cargo'] = df_tjba['pje_ds_orgao_julgador_cargo'].str.upper()
df_tjba['pje_ds_natureza'] = df_tjba['pje_ds_natureza'].str.upper()
#df_tjba['vm_ds_prioridade'] = df_tjba['vm_ds_prioridade'].str.upper()

# Retirando o timestamp
#df_tjba['vm_dt_recebimento'] = df_tjba['vm_dt_recebimento'].dt.date
#df_tjba['vm_dt_recebimento'] = pd.to_datetime(df_tjba['vm_dt_recebimento']) # Erro 1001-10-30, at position 93295

df_tjba['vm_dt_movimento'] = df_tjba['vm_dt_movimento'].dt.date
df_tjba['vm_dt_movimento'] = pd.to_datetime(df_tjba['vm_dt_movimento'])

In [23]:
# Percorre todas as colunas do df_tjba do tipo object, retirando espaços laterais e mais de um espaços entre palavras
for col in df_tjba.select_dtypes(include=['object']).columns:
    df_tjba[col] = (
        df_tjba[col].str.strip().str.replace(r'\s+', ' ', regex=True)
	)

In [26]:
df_tjba.head()

,vm_processo,vm_dt_recebimento,vm_dt_movimento,vm_cod_comarca,vm_comarca,vm_cod_vara,vm_vara,vm_cod_cls_cnj,vm_classe,pje_ds_natureza,vm_movimento,pje_id_competencia,vm_competencia,vm_mapa_magistrado,vm_magistrado,vm_consulta,vm_situacao,pje_vl_peso_processual,vm_sistema
0,8003619-75.2022.8.05.0271,2022-09-19 14:02:01+00:00,2024-08-26,1255,VALENCA,11534,2ª VARA DE FEITOS DE REL DE CONS. CÍVEL E COMERCIAIS,65,ACAO CIVIL PUBLICA,CONHECIMENTO,PROFERIDO DESPACHO DE MERO EXPEDIENTE,9.0,FAZENDA PUBLICA ATOS ADMINISTRATIVOS,378.0,LEONARDO RULIAN CUSTODIO,DESPACHO,EM ANDAMENTO,4.0,PJEPG
1,8003068-15.2022.8.05.0039,2022-02-06 18:58:24+00:00,2024-09-19,115,CAMACARI,11453,2ª VARA DA FAZENDA PÚBLICA,1116,EXECUCAO FISCAL,EXECUÇÃO,EXTINTO O PROCESSO POR AUSENCIA DAS CONDICOES DA ACAO,6.0,FAZENDA PUBLICA TRIBUTOS,166.0,DANIEL LIMA FALCAO,JULGAMENTO,BAIXADO,4.0,PJEPG
2,0088391-11.2003.8.05.0001,2003-07-18 13:10:09+00:00,2025-07-18,18,SALVADOR,111452,15ª VARA DA FAZENDA PÚBLICA,1116,EXECUCAO FISCAL,EXECUÇÃO,PROFERIDO DESPACHO DE MERO EXPEDIENTE,9.0,FAZENDA PUBLICA ATOS ADMINISTRATIVOS,161.0,CRISTIANE MENEZES SANTOS BARRETO,DESPACHO,EM ANDAMENTO,4.0,PJEPG
3,0000087-73.2019.8.05.0066,2019-03-18 18:20:00+00:00,2025-07-09,1165,CONDEUBA,11754,VARA JURISDIÇÃO PLENA,283,ACAO PENAL PROCEDIMENTO ORDINARIO,CONHECIMENTO,PROFERIDO DESPACHO DE MERO EXPEDIENTE,53.0,CRIME,1492.0,CARLOS TIAGO SILVA ADAES,DESPACHO,EM ANDAMENTO,4.0,PJEPG
4,8055437-37.2021.8.05.0001,2021-05-28 13:47:59+00:00,2025-05-30,18,SALVADOR,11553,6ª VARA CÍVEL E COMERCIAL,7,PROCEDIMENTO COMUM CIVEL,CONHECIMENTO,PROFERIDO DESPACHO DE MERO EXPEDIENTE,12.0,CIVEL,750.0,CARLOS CARVALHO RAMOS DE CERQUEIRA JUNIOR,DESPACHO,EM ANDAMENTO,4.0,PJEPG


In [27]:
df_tjba.shape

(11715246, 19)

In [ ]:
#nomes_magistrados = df_tjba['vm_magistrado'].dropna().unique()
#
#df_magistrados['vm_nome'] = [
#    next((nome for nome in nomes_magistrados if str(nome).startswith(str(cadastro))), None) 
#    for cadastro in df_magistrados['Nome']
#]

In [ ]:
## Agora com os nomes ajustados podemos colocando a matrícula dos magistrados 
## que temos no cadastro no df principal
#df_tjba = df_tjba.merge(
#    df_magistrados[['Nome', 'Matrícula Funcional']], 
#    left_on='vm_magistrado', 
#    right_on='Nome', 
#    how='left'
#).rename(columns={'Matrícula Funcional': 'pt_matricula'})
#
## Removendo coluna Nome
#df_tjba.drop('Nome', axis=1, inplace=True)
#
## Pegando lista de colunas
#cols = list(df_tjba.columns)
#
## Removendo coluna
#cols.remove('pt_matricula')
#
## Achando posição, onde adicionar coluna
#pos = cols.index('vm_mapa_magistrado') + 1
#
## Adicionado coluna na posição escolhida
#cols.insert(pos, 'pt_matricula')
#
## Reordenado df
#df_tjba = df_tjba[cols]

# Análise Exploratória - Conjunto de Dados Final

In [28]:
# Quantidade de registro por sistema df TJBA
df_tjba['vm_sistema'].value_counts()

vm_sistema
PJEPG     11556834
SEEU        158333
SAJPG           71
SAIPRO           8
Name: count, dtype: int64

In [29]:
# Verificando pessos dos processos df TJBA
df_tjba['pje_vl_peso_processual'].value_counts()

pje_vl_peso_processual
4.0    11537248
2.0       19571
1.0          15
Name: count, dtype: int64

In [30]:
# Verifica quantidade de processos por consulta
df_tjba['vm_consulta'].value_counts()

vm_consulta
DESPACHO       3223440
ACERVO         1957071
DECISAO        1838819
BAIXADO        1726833
JULGAMENTO     1621368
DISTRIBUIDO    1347715
Name: count, dtype: int64

Um processo que recebeu peso 2 e foi julgado 5 vezes pelo mesmo magistrado, eu devo contabilizar 10 ou somente 2?

*	**Evitar Superestimação da Produtividade:** Se um processo de peso 2 é julgado 5 vezes pelo mesmo magistrado, e contabiliza 10 pontos (5 x 2), estaria inflando a produtividade desse juiz. O processo, em sua essência, ainda é um único processo de peso 2. O trabalho adicional nas 4 sentenças subsequentes é parte do esforço para finalizar aquele processo, mas não representa 5 processos distintos de peso 2.

*	**Coerência com o Conceito de Peso:** O peso é atribuído ao processo como um todo, refletindo sua complexidade intrínseca. Ele não é um valor que se multiplica a cada interação, mas sim um valor que é "consumido" quando o processo é finalmente resolvido.

In [34]:
# Seleciona dados de julgamentos de 2024
df_julgados = df_tjba[(df_tjba['vm_consulta'] == 'JULGAMENTO') & 
                    (df_tjba['vm_dt_movimento'].dt.year == 2024) & 
                    (df_tjba['vm_sistema'] == 'PJEPG')].drop_duplicates(['vm_processo', 'vm_mapa_magistrado'])

# Convertendo float para int
df_julgados['pje_vl_peso_processual'] = df_julgados['pje_vl_peso_processual'].astype('Int64')
df_julgados['vm_mapa_magistrado'] = df_julgados['vm_mapa_magistrado'].astype('Int64')

# Criando campo para identificação dos magistrados
df_julgados['juiz_str'] = 'JUIZ_' + df_julgados['vm_mapa_magistrado'].astype(str)
df_julgados['vara_str'] = 'VARA_' + df_julgados['vm_cod_vara'].astype(str)

df_julgados.head()

,vm_processo,vm_dt_recebimento,vm_dt_movimento,vm_cod_comarca,vm_comarca,vm_cod_vara,vm_vara,vm_cod_cls_cnj,vm_classe,pje_ds_natureza,vm_movimento,pje_id_competencia,vm_competencia,vm_mapa_magistrado,vm_magistrado,vm_consulta,vm_situacao,pje_vl_peso_processual,vm_sistema,juiz_str,vara_str
1,8003068-15.2022.8.05.0039,2022-02-06 18:58:24+00:00,2024-09-19,115,CAMACARI,11453,2ª VARA DA FAZENDA PÚBLICA,1116,EXECUCAO FISCAL,EXECUÇÃO,EXTINTO O PROCESSO POR AUSENCIA DAS CONDICOES DA ACAO,6.0,FAZENDA PUBLICA TRIBUTOS,166,DANIEL LIMA FALCAO,JULGAMENTO,BAIXADO,4,PJEPG,JUIZ_166,VARA_11453
39,0005113-64.2006.8.05.0274,2006-05-17 18:53:25+00:00,2024-02-27,1251,VITORIA DA CONQUISTA,11426,4ª VARA DE FEITOS DE REL DE CONS. CÍVEL E COMERCIAIS,12154,EXECUCAO DE TITULO EXTRAJUDICIAL,EXECUÇÃO,EXTINTO O PROCESSO POR INEXISTENCIA DE BENS PENHORAVEIS,12.0,CIVEL,432,MARCIA DA SILVA ABREU,JULGAMENTO,BAIXADO,4,PJEPG,JUIZ_432,VARA_11426
49,0756964-32.2015.8.05.0001,2015-03-05 21:40:03+00:00,2024-09-10,18,SALVADOR,1115,13ª VARA DA FAZENDA PÚBLICA,1116,EXECUCAO FISCAL,EXECUÇÃO,EXTINTO O PROCESSO POR AUSENCIA DAS CONDICOES DA ACAO,46.0,FAZENDA PUBLICA EXECUCAO MUNICIPIO,453,MARIA CRISTINA LADEIA DE SOUZA,JULGAMENTO,BAIXADO,4,PJEPG,JUIZ_453,VARA_1115
64,8002466-95.2021.8.05.0156,2021-12-01 17:02:27+00:00,2024-05-10,178,MACAUBAS,11735,VARA CRIME,283,ACAO PENAL PROCEDIMENTO ORDINARIO,CONHECIMENTO,JULGADO IMPROCEDENTE O PEDIDO,53.0,CRIME,1495,DANILLO AUGUSTO GOMES DE MOURA E SILVA,JULGAMENTO,BAIXADO,4,PJEPG,JUIZ_1495,VARA_11735
71,8044320-49.2021.8.05.0001,2021-05-03 17:54:01+00:00,2024-07-12,18,SALVADOR,11501,19ª VARA DE RELACOES DE CONSUMO,7,PROCEDIMENTO COMUM CIVEL,CONHECIMENTO,HOMOLOGADA A TRANSACAO,14.0,RELACOES DE CONSUMO,264,GLAUTEMBERG BASTOS DE LUNA,JULGAMENTO,BAIXADO,4,PJEPG,JUIZ_264,VARA_11501


In [35]:
df_julgados.shape

(1070561, 21)

In [36]:
# Agrupar e contar juízes que atuam no mesmo processo
processos_multi_juiz = df_julgados.groupby('vm_processo')['vm_mapa_magistrado'].nunique()
processos_com_multiplos_juizes = processos_multi_juiz[processos_multi_juiz > 1]

print(f"Processos com múltiplos juízes: {len(processos_com_multiplos_juizes)}\n")
print(processos_com_multiplos_juizes.head())

Processos com múltiplos juízes: 4401

vm_processo
0000001-54.2014.8.05.0171    2
0000002-46.1992.8.05.0030    2
0000008-77.1997.8.05.0030    2
0000009-15.2018.8.05.0034    2
0000010-47.1997.8.05.0030    2
Name: vm_mapa_magistrado, dtype: int64


In [38]:
# Quantidade de magistrados atuantes em 2024
qtd_magistrados = df_tjba[(df_tjba['vm_dt_movimento'].dt.year == 2024)]['vm_magistrado'].nunique()

# Quantidade de processos julgados em 2024
qtd_julgados = df_julgados.shape[0]

# Índice de Produtividade dos Magistrados
ipm = qtd_julgados / qtd_magistrados

print('Quantidade de Magistrados Atuantes: ', qtd_magistrados)
print('Quantidade de Julgamentos: ', qtd_julgados)
print('Índice de Produtividade dos Magistrados : ', round(ipm, 2))

Quantidade de Magistrados Atuantes:  629
Quantidade de Julgamentos:  1070561
Índice de Produtividade dos Magistrados :  1702.0


In [39]:
df_julgados[df_julgados['vm_processo'] == '0000001-54.2014.8.05.0171']

,vm_processo,vm_dt_recebimento,vm_dt_movimento,vm_cod_comarca,vm_comarca,vm_cod_vara,vm_vara,vm_cod_cls_cnj,vm_classe,pje_ds_natureza,vm_movimento,pje_id_competencia,vm_competencia,vm_mapa_magistrado,vm_magistrado,vm_consulta,vm_situacao,pje_vl_peso_processual,vm_sistema,juiz_str,vara_str
1705118,0000001-54.2014.8.05.0171,2014-01-02 11:46:00+00:00,2024-11-22,1228,ANDARAI,11685,VARA JURISDIÇÃO PLENA,283,ACAO PENAL PROCEDIMENTO ORDINARIO,CONHECIMENTO,JULGADO PROCEDENTE EM PARTE O PEDIDO,53.0,CRIME,1491,CAMILA SOUSA PINTO DE ABREU,JULGAMENTO,JULGADO,4,PJEPG,JUIZ_1491,VARA_11685
10538699,0000001-54.2014.8.05.0171,2014-01-02 11:46:00+00:00,2024-12-16,1228,ANDARAI,11685,VARA JURISDIÇÃO PLENA,283,ACAO PENAL PROCEDIMENTO ORDINARIO,CONHECIMENTO,EMBARGOS DE DECLARACAO ACOLHIDOS EM PARTE,53.0,CRIME,1505,GESSICA OLIVEIRA SANTOS,JULGAMENTO,JULGADO,4,PJEPG,JUIZ_1505,VARA_11685


In [40]:
# Verificando se temos magistrados null 
df_julgados['vm_magistrado'].isna().sum()

np.int64(0)

In [41]:
# Verifica a incidência de porcessos julgados mais de uma vez
df_julgados['vm_processo'].value_counts()[lambda x: x > 1]

vm_processo
8000644-58.2022.8.05.0149    3
8012831-61.2023.8.05.0150    3
8010821-44.2023.8.05.0150    3
8000041-29.2021.8.05.0081    3
0806959-82.2013.8.05.0001    3
                            ..
8001198-61.2021.8.05.0073    2
8008905-21.2022.8.05.0146    2
0786930-74.2014.8.05.0001    2
8164703-22.2022.8.05.0001    2
0000486-82.2016.8.05.0139    2
Name: count, Length: 4401, dtype: int64

In [43]:
# Verifica tipo de consultas
print('Tipos de consulta: ', df_julgados['vm_consulta'].unique())
print('Quantidade de magistrados: ',df_julgados['vm_magistrado'].nunique())
print('Quantidade de registros', df_julgados.shape[0])
print('Quantidade de processos', df_julgados['vm_processo'].nunique())

Tipos de consulta:  ['JULGAMENTO']
Quantidade de magistrados:  561
Quantidade de registros 1070561
Quantidade de processos 1066148


In [44]:
# Gerando df com quantidade de magistrados por Comarca
df_comarca = (
    df_julgados.groupby(['vm_cod_comarca', 'vm_comarca'])['vm_magistrado'].nunique().reset_index(name='Qtd. Magistrado')
).rename(columns={
    'vm_cod_comarca': 'Cod. Comarca',
    'vm_comarca': 'Comarca'
})

df_comarca


,Cod. Comarca,Comarca,Qtd. Magistrado
0,18,SALVADOR,267
1,110,PORTO SEGURO,18
2,111,EUNAPOLIS,8
3,112,ITANHEM,3
4,114,BARRA,5
...,...,...,...
198,1253,ITAPETINGA,6
199,1254,ITABUNA,18
200,1255,VALENCA,9
201,1256,IPIAU,6


In [ ]:
# Comarca com maior números de magistrados
df_comarca[df_comarca['Qtd. Magistrado'] == df_comarca['Qtd. Magistrado'].max()]

,Cod. Comarca,Comarca,Qtd. Magistrado
0,18,SALVADOR,267


In [48]:
# Média de processos julgados no dia por juiz
df_mean_j = df_julgados.groupby([
    'juiz_str', 
    df_julgados['vm_dt_movimento'].dt.date
]).size().groupby('juiz_str').mean().round(2).reset_index()

df_mean_j.columns = ['Cod. Juiz', 'Med. Dia']
df_mean_j

,Cod. Juiz,Med. Dia
0,JUIZ_10,2.63
1,JUIZ_1001,5.39
2,JUIZ_102,1.70
3,JUIZ_103,8.67
4,JUIZ_104,6.28
...,...,...
556,JUIZ_943,14.98
557,JUIZ_958,13.50
558,JUIZ_96,3.11
559,JUIZ_98,4.90


In [49]:
# Juiz com maior média de julgamentos
df_mean_j[df_mean_j['Med. Dia'] == df_mean_j['Med. Dia'].max()]

,Cod. Juiz,Med. Dia
194,JUIZ_166,246.93


In [56]:
df_julgados[df_julgados['juiz_str'] == 'JUIZ_166'].sort_values('vm_dt_movimento', ascending=False).head()

,vm_processo,vm_dt_recebimento,vm_dt_movimento,vm_cod_comarca,vm_comarca,vm_cod_vara,vm_vara,vm_cod_cls_cnj,vm_classe,pje_ds_natureza,vm_movimento,pje_id_competencia,vm_competencia,vm_mapa_magistrado,vm_magistrado,vm_consulta,vm_situacao,pje_vl_peso_processual,vm_sistema,juiz_str,vara_str
11702418,8005615-57.2024.8.05.0039,2024-05-16 23:30:50+00:00,2024-12-31,115,CAMACARI,11453,2ª VARA DA FAZENDA PÚBLICA,14695,PROCEDIMENTO DO JUIZADO ESPECIAL DA FAZENDA PUBLICA,CONHECIMENTO,JULGADO PROCEDENTE EM PARTE O PEDIDO,9.0,FAZENDA PUBLICA ATOS ADMINISTRATIVOS,166,DANIEL LIMA FALCAO,JULGAMENTO,EM GRAU DE RECURSO,4,PJEPG,JUIZ_166,VARA_11453
7750831,8005143-56.2024.8.05.0039,2024-05-07 16:59:54+00:00,2024-12-31,115,CAMACARI,11453,2ª VARA DA FAZENDA PÚBLICA,14695,PROCEDIMENTO DO JUIZADO ESPECIAL DA FAZENDA PUBLICA,CONHECIMENTO,JULGADO PROCEDENTE EM PARTE O PEDIDO,9.0,FAZENDA PUBLICA ATOS ADMINISTRATIVOS,166,DANIEL LIMA FALCAO,JULGAMENTO,EM GRAU DE RECURSO,4,PJEPG,JUIZ_166,VARA_11453
9012844,8005614-72.2024.8.05.0039,2024-05-16 23:27:52+00:00,2024-12-31,115,CAMACARI,11453,2ª VARA DA FAZENDA PÚBLICA,14695,PROCEDIMENTO DO JUIZADO ESPECIAL DA FAZENDA PUBLICA,CONHECIMENTO,JULGADO PROCEDENTE EM PARTE O PEDIDO,9.0,FAZENDA PUBLICA ATOS ADMINISTRATIVOS,166,DANIEL LIMA FALCAO,JULGAMENTO,EM GRAU DE RECURSO,4,PJEPG,JUIZ_166,VARA_11453
10663965,8005608-65.2024.8.05.0039,2024-05-16 22:38:12+00:00,2024-12-31,115,CAMACARI,11453,2ª VARA DA FAZENDA PÚBLICA,14695,PROCEDIMENTO DO JUIZADO ESPECIAL DA FAZENDA PUBLICA,CONHECIMENTO,JULGADO PROCEDENTE EM PARTE O PEDIDO,9.0,FAZENDA PUBLICA ATOS ADMINISTRATIVOS,166,DANIEL LIMA FALCAO,JULGAMENTO,EM GRAU DE RECURSO,4,PJEPG,JUIZ_166,VARA_11453
11171621,8013786-37.2023.8.05.0039,2023-12-19 13:03:35+00:00,2024-12-31,115,CAMACARI,11453,2ª VARA DA FAZENDA PÚBLICA,14695,PROCEDIMENTO DO JUIZADO ESPECIAL DA FAZENDA PUBLICA,CONHECIMENTO,JULGADO PROCEDENTE EM PARTE O PEDIDO,15.0,JUIZADOS ESPECIAIS DA FAZENDA PUBLICA,166,DANIEL LIMA FALCAO,JULGAMENTO,EM GRAU DE RECURSO,4,PJEPG,JUIZ_166,VARA_11453


In [ ]:
# IPM - Índice de Produtividade de Magistrados (JULGADOS) em pesos
sum_pesos = df_julgados['pje_vl_peso_processual'].sum()
ipm_juldados = sum_pesos / qtd_julgados

print('IPM Julgados em pesos: ', round(ipm_juldados, 4))

IPM Julgados em pesos:  3.9945


# Modelo

A Lógica Central do Modelo:

Imagine o modelo como um "planejador inteligente" que tem um objetivo claro: maximizar o número total de processos julgados (considerando seus pesos/dificuldade) no tribunal como um todo.

*	O que o modelo "vê":
	*	Sobre os Juízes: Quem são, qual a produtividade de cada um (quantos "pesos" de processos conseguem julgar por trimestre), e em quais áreas são especializados (cível criminal, família, etc.)
	*	Sobre as Varas: Onde estão, qual a capacidade máxima de cada uma, qual a demanda de processos que chega (por tipo), e quantos juízes cada vara pode comportar
	*	As Regras: Juízes só podem trabalhar onde têm especialização, cada vara tem limite de capacidade, etc.

## Funcionamento

Atualmente o modelo foca em maximizar a produtividade total $P_i$ dos juízes, respeitando a especialização $E_{i,k}$ a capacidade das varas $C_j$, a demanda $D_{j,k}$ e o número máximo de juízes por vara $M_{j,t}$.

O modelo está preparado para UM período por vez. Por que trimestral? 

Com apenas um ano de dados, o período trimestral (4 pontos de dados por juiz) oferece uma medida de produtividade mais estável e representativa. Ele suaviza flutuações de curto prazo (como férias ou casos complexos que levam mais tempo) que poderiam distorcer a produtividade mensal. O dado trimestral é menos suscetível a ruídos e dá uma visão mais consistente da capacidade do juiz.

Como está formulado:
*	As variáveis de decisão são $x_{i,j}$ (sem índice temporal)
*	Os parâmetros têm índice temporal $P_{i,t} \ C_{j,t} \ D_{j,k,t}$ mas usamos apenas um período de referência
*	As restrições são formuladas para um período específico
<br></br>

*	Opção 1: Utilizar um período específico (Atual)
	*	Selecionar apenas um trimestre (ex: T4 de 2024) para otimização
	*	Usar os dados desse trimestre como base para a alocação futura	
	*	Vantagens: Modelo mais simples, focado, e computacionalmente mais eficiente
	*	Quando usar: Quando você quer otimizar a alocação para o próximo período baseado no desempenho mais recente

*	Opção 2: Usar todos os períodos simultaneamente (requer adaptação do modelo)
	*	Incluir T1, T2, T3 e T4 no modelo
	*	Otimizar considerando a variação temporal
	*	Vantagens: Captura sazonalidade e variações ao longo do ano
	*	Desvantagens: Modelo muito mais complexo, com mais variáveis e restrições
<br></br>

Por que isso funciona com $P_{i,t}$ (Produtividade Ponderada)?

O modelo não precisa saber o "tempo" exato. Ele trabalha com a capacidade de entrega do juiz, medida em "pontos de processo".
<br></br>

Por que a abordagem por PESOS é vantajosa sobre a simples QUANTIDADE de processos?

*	Reflete a Complexidade: Reconhece que nem todo processo exige o mesmo esforço. Um processo de família complexo não pode ser comparado a um simples processo de trânsito em termos de trabalho.

*	Alocação Mais Justa: Permite que o modelo aloque juízes considerando não apenas quantos processos eles podem julgar, mas também a dificuldade desses processos. Isso leva a uma distribuição de carga de trabalho mais equitativa.

## Conjuntos

$$I: \text{ Conjunto de todos os juízes disponíveis para alocação. Cada juiz } i \in I \text{ é um elemento único}$$

$$ \text{Exemplo: } I = \{ \text{Juiz}_1, \ \text{Juiz}_2, \ \dots, \ \text{Juiz}_n \}$$

$$J: \text{ Conjunto de todas as varas do 1º Grau. Cada vara } j \in J \text{ é uma unidade de destino para alocação de juízes} $$

$$ \text{Exemplo: } J = \{ \text{Vara}_1, \ \text{Vara}_2, \ \dots, \ \text{Vara}_n \}$$

$$K: \text{Conjunto de todas as compet6encias das varas (e,consequentemente, das especializações dos juízes). Cada natureza } k \in K \text{ representa uma área do direito.}$$

$$ \text{Exemplo: } K = \{ \text{Cível, Criminal, Fazenda Pública, Família, ...} \}$$

## Parâmetros



( $P_{i,t}$ ) -  Produtividade ponderada do juiz no período $i \in I, \text{no período } t$. Representa a soma dos pesos dos processos julgados pelo juiz $i$ no trimestre $t$. Este parâmetro é crucial para medir a capacidade de trabalho real de cada magistrado.

In [67]:
P_it = df_julgados[['juiz_str']].drop_duplicates().reset_index(drop=True)

# Criando produtivedade ponderada para 1º trimestre
P_it['T1'] = P_it['juiz_str'].map(
    df_julgados[df_julgados['vm_dt_movimento'].dt.quarter == 1]
    .groupby('juiz_str')['pje_vl_peso_processual'].sum()
).fillna(0)

# Criando produtivedade ponderada para 2º trimestre
P_it['T2'] = P_it['juiz_str'].map(
    df_julgados[df_julgados['vm_dt_movimento'].dt.quarter == 2]
    .groupby('juiz_str')['pje_vl_peso_processual'].sum()
).fillna(0)

# Criando produtivedade ponderada para 3º trimestre
P_it['T3'] = P_it['juiz_str'].map(
    df_julgados[df_julgados['vm_dt_movimento'].dt.quarter == 3]
    .groupby('juiz_str')['pje_vl_peso_processual'].sum()
).fillna(0)

# Criando produtivedade ponderada para 4º trimestre
P_it['T4'] = P_it['juiz_str'].map(
    df_julgados[df_julgados['vm_dt_movimento'].dt.quarter == 4]
    .groupby('juiz_str')['pje_vl_peso_processual'].sum()
).fillna(0)

P_it

,juiz_str,T1,T2,T3,T4
0,JUIZ_166,6960,30896,55840,71254
1,JUIZ_432,974,1582,1260,904
2,JUIZ_453,28540,15180,15740,11676
3,JUIZ_1495,468,832,1868,3046
4,JUIZ_264,1476,2026,4010,2034
...,...,...,...,...,...
556,JUIZ_651,4,0,0,0
557,JUIZ_575,4,0,0,0
558,JUIZ_71,4,0,0,0
559,JUIZ_38,0,8,0,0


( $E_{k,i}$ ) - Parâmetro binário que indica se o juiz $i \in I$  possui especialização na natureza $k \in K.E_{i,k} = 1$ se o juiz é especializado na natureza $k$, e $0$ caso contrário. A especialização é fundamental para garantir que os juízes sejam alocados a varas onde sua expertise é relevante.

In [68]:
E_ik = df_julgados[['juiz_str']].drop_duplicates().reset_index(drop=True)
naturezas = df_julgados[['vm_competencia']].drop_duplicates().reset_index(drop=True)
E_ik = E_ik.merge(naturezas, how='cross')

E_ik = E_ik.merge(
    df_julgados[['juiz_str', 'vm_competencia']].drop_duplicates(),
    on=['juiz_str', 'vm_competencia'],
    how='left',
    indicator=True
)

E_ik['flag'] = (E_ik['_merge'] == 'both').astype(int)
E_ik = E_ik.drop(columns=['_merge'], axis=1)

E_ik

,juiz_str,vm_competencia,flag
0,JUIZ_166,FAZENDA PUBLICA TRIBUTOS,1
1,JUIZ_166,CIVEL,1
2,JUIZ_166,FAZENDA PUBLICA EXECUCAO MUNICIPIO,1
3,JUIZ_166,CRIME,0
4,JUIZ_166,RELACOES DE CONSUMO,0
...,...,...,...
54973,JUIZ_268,[CEJUSC PREPROCESSUAL FAMILIA] ZONA 18,0
54974,JUIZ_268,FAMILIA RECESSO SALVADOR,0
54975,JUIZ_268,INFANCIA E JUVENTUDE RECESSO SALVADOR,0
54976,JUIZ_268,CRIMINAL RECESSO SALVADOR,0


( $C_{i,j}$ ) - Capacidade máxima ponderada da vara $j \in J, \text{no período } t$. Representa o volume máximo de processos (em pontos) que a vara $j$ demonstrou ser capaz de julgar no trimestre $t$. Este parâmetro atua como um limite superior para a carga de trabalho que uma vara pode absorver.

In [69]:
C_jt = df_julgados[['vara_str', 'vm_vara']].drop_duplicates().reset_index(drop=True)

# Criando capacidade ponderada para 1º trimestre
C_jt['T1'] = C_jt['vara_str'].map(
    df_julgados[df_julgados['vm_dt_movimento'].dt.quarter == 1]
    .groupby('vara_str')['pje_vl_peso_processual'].sum()
).fillna(0)

# Criando capacidade ponderada para 2º trimestre
C_jt['T2'] = C_jt['vara_str'].map(
    df_julgados[df_julgados['vm_dt_movimento'].dt.quarter == 2]
    .groupby('vara_str')['pje_vl_peso_processual'].sum()
).fillna(0)

# Criando capacidade ponderada para 3º trimestre
C_jt['T3'] = C_jt['vara_str'].map(
    df_julgados[df_julgados['vm_dt_movimento'].dt.quarter == 3]
    .groupby('vara_str')['pje_vl_peso_processual'].sum()
).fillna(0)

# Criando capacidade ponderada para 4º trimestre
C_jt['T4'] = C_jt['vara_str'].map(
    df_julgados[df_julgados['vm_dt_movimento'].dt.quarter == 4]
    .groupby('vara_str')['pje_vl_peso_processual'].sum()
).fillna(0)

C_jt

,vara_str,vm_vara,T1,T2,T3,T4
0,VARA_11453,2ª VARA DA FAZENDA PÚBLICA,6948,30896,55820,71262
1,VARA_11426,4ª VARA DE FEITOS DE REL DE CONS. CÍVEL E COMERCIAIS,970,1578,1260,908
2,VARA_1115,13ª VARA DA FAZENDA PÚBLICA,28540,15564,15736,11672
3,VARA_11735,VARA CRIME,228,480,236,616
4,VARA_11501,19ª VARA DE RELACOES DE CONSUMO,1984,3226,4476,2824
...,...,...,...,...,...,...
790,VARA_11907,VARA DE RECESSO JUDICIAL,4,0,0,0
791,VARA_11905,VARA DE RECESSO JUDICIAL,0,0,0,4
792,VARA_11922,PLANTÃO JUDICIÁRIO 1º GRAU,0,0,0,4
793,VARA_111163,VARA JURISDIÇÃO PLENA,4,0,0,0


( $D_{j,k,t}$ ) - Demanda ponderada de processos para a vara $j \in J \text{ na competência } k \in K \text{ no período } t$. Representa a soma dos pesos dos novos processos que chegam à vara $j \text{ com a natureza } k \text{ no trimestre } t$. Este parâmetro reflete a necessidade de ttrabalho de cada vara, segmentada por especialização.

In [70]:
D_jkt = df_julgados[['vara_str']].drop_duplicates().reset_index(drop=True)
vara = df_julgados[['vm_competencia']].drop_duplicates().reset_index(drop=True)
D_jkt = D_jkt.merge(vara, how='cross')

# Criar lookup table
lookup_q1 = df_julgados[
    df_julgados['vm_dt_movimento'].dt.quarter == 1
].set_index(['vara_str', 'vm_competencia'])['pje_vl_peso_processual'].groupby(level=[0,1]).sum()

lookup_q2 = df_julgados[
    df_julgados['vm_dt_movimento'].dt.quarter == 2
].set_index(['vara_str', 'vm_competencia'])['pje_vl_peso_processual'].groupby(level=[0,1]).sum()

lookup_q3 = df_julgados[
    df_julgados['vm_dt_movimento'].dt.quarter == 3
].set_index(['vara_str', 'vm_competencia'])['pje_vl_peso_processual'].groupby(level=[0,1]).sum()

lookup_q4 = df_julgados[
    df_julgados['vm_dt_movimento'].dt.quarter == 4
].set_index(['vara_str', 'vm_competencia'])['pje_vl_peso_processual'].groupby(level=[0,1]).sum()

# Transform usando map no índice combinado
# Criando produtivedade ponderada para trimestres
D_jkt['T1'] = pd.MultiIndex.from_arrays([D_jkt['vara_str'], D_jkt['vm_competencia']]).map(lookup_q1).fillna(0)
D_jkt['T2'] = pd.MultiIndex.from_arrays([D_jkt['vara_str'], D_jkt['vm_competencia']]).map(lookup_q2).fillna(0)
D_jkt['T3'] = pd.MultiIndex.from_arrays([D_jkt['vara_str'], D_jkt['vm_competencia']]).map(lookup_q3).fillna(0)
D_jkt['T4'] = pd.MultiIndex.from_arrays([D_jkt['vara_str'], D_jkt['vm_competencia']]).map(lookup_q4).fillna(0)

D_jkt

,vara_str,vm_competencia,T1,T2,T3,T4
0,VARA_11453,FAZENDA PUBLICA TRIBUTOS,2068,5220,11560,29564
1,VARA_11453,CIVEL,40,152,236,1190
2,VARA_11453,FAZENDA PUBLICA EXECUCAO MUNICIPIO,4164,24856,42868,37264
3,VARA_11453,CRIME,0,0,0,0
4,VARA_11453,RELACOES DE CONSUMO,0,0,0,0
...,...,...,...,...,...,...
77905,VARA_11945,[CEJUSC PREPROCESSUAL FAMILIA] ZONA 18,0,0,0,0
77906,VARA_11945,FAMILIA RECESSO SALVADOR,0,0,0,0
77907,VARA_11945,INFANCIA E JUVENTUDE RECESSO SALVADOR,0,0,0,0
77908,VARA_11945,CRIMINAL RECESSO SALVADOR,0,0,0,0


( $M_{j,t}$ ) - Número máximo de juízes que podem ser alocados à vara $j \in J \text{ no período } t$. Este parâmetro representa uma estimativa da capacidade máxima da vara baseada no maior número de juízes que atuaram nela durante os trimestres de 2024.

O total representa quantos juízes diferentes passaram pela vara ao longo do ano, não o máximo simultâneo. Para o modelo de otimização, faz mais sentido usar o maior valor trimestral como capacidade máxima:

In [71]:
# Gerando df com quantidade de magistrados por Vara/Trimestre (M_jt)
M_jt = (
    df_julgados.groupby(['vara_str', 'vm_vara'])['vm_magistrado']
    .nunique().reset_index(name='Qtd. Total')
)

#.rename(columns={
#    'vara_str': 'Cod. Vara',
#    'vm_vara': 'Vara'
#})

# Primeiro trimestre
M_jt['T1'] = M_jt['vara_str'].map(
    df_julgados[df_julgados['vm_dt_movimento'].dt.quarter == 1]
    .groupby('vara_str')['juiz_str'].nunique()
).fillna(0).astype(int)

# Segundo trimestre
M_jt['T2'] = M_jt['vara_str'].map(
    df_julgados[df_julgados['vm_dt_movimento'].dt.quarter == 2]
    .groupby('vara_str')['juiz_str'].nunique()
).fillna(0).astype(int)

# Terceiro trimestre
M_jt['T3'] = M_jt['vara_str'].map(
    df_julgados[df_julgados['vm_dt_movimento'].dt.quarter == 3]
    .groupby('vara_str')['juiz_str'].nunique()
).fillna(0).astype(int)

# Quarto trimestre
M_jt['T4'] = M_jt['vara_str'].map(
    df_julgados[df_julgados['vm_dt_movimento'].dt.quarter == 4]
    .groupby('vara_str')['juiz_str'].nunique()
).fillna(0).astype(int)

M_jt

,vara_str,vm_vara,Qtd. Total,T1,T2,T3,T4
0,VARA_111005,VARA RECESSO CIVEL DE SERRINHA,1,0,0,0,1
1,VARA_111047,VARA CÍVEL,1,0,0,0,1
2,VARA_111057,VARA CÍVEL,1,0,0,0,1
3,VARA_11109,VARA JURISDIÇÃO PLENA,2,1,2,2,2
4,VARA_11110,VARA JURISDIÇÃO PLENA,9,1,1,1,9
...,...,...,...,...,...,...,...
790,VARA_11948,VARA RECESSO CIVEL DE JACOBINA,1,1,0,0,0
791,VARA_11953,VARA RECESSO CIVEL DE PORTO SEGURO,1,1,0,0,1
792,VARA_11959,PLANTÃO JUDICIÁRIO VITÓRIA DA CONQUISTA,1,1,0,0,0
793,VARA_11967,VARA CRIME,1,0,0,0,1


( $N_{i,t}$ ) - Número de varas distintas em que o juiz $i \in I \text{ já atuou no período } t$. Esta informação é valiosa para análises adicionais e para entender a flexibilidade ou a experiência de cada juiz ao longo do tempo. Pode ser incorporada em restrições mais avançadas (ex: preferência por juízes com maior $N_{i,t}$ para varas com alta variabilidade de demanda, ou como um custo de realocação para juízes com baixo $N_{i,t}$). Embora não seja um parâmetro diretamente usado nas restrições básicas do modelo de alocação, sua granularidade trimestral a torna ainda mais útil para análises de flexibilidade e potencial de realocação ao longo do tempo.

In [72]:
# Verificando a quantidade varas que cada juiz atua por trimestre (N_it)
N_it = df_julgados.groupby('juiz_str')['vara_str'].nunique().reset_index().sort_values('vara_str', ascending=False)
N_it.columns = ['Cod. Juiz', 'Qtd. Vara']

# Primeiro trimestre
N_it['T1'] = N_it['Cod. Juiz'].map(
    df_julgados[df_julgados['vm_dt_movimento'].dt.quarter == 1]
    .groupby('juiz_str')['vara_str'].nunique()
).fillna(0).astype(int)

# Segundo trimestre
N_it['T2'] = N_it['Cod. Juiz'].map(
    df_julgados[df_julgados['vm_dt_movimento'].dt.quarter == 2]
    .groupby('juiz_str')['vara_str'].nunique()
).fillna(0).astype(int)

# Terceiro trimestre
N_it['T3'] = N_it['Cod. Juiz'].map(
    df_julgados[df_julgados['vm_dt_movimento'].dt.quarter == 3]
    .groupby('juiz_str')['vara_str'].nunique()
).fillna(0).astype(int)

# Quarto trimestre
N_it['T4'] = N_it['Cod. Juiz'].map(
    df_julgados[df_julgados['vm_dt_movimento'].dt.quarter == 4]
    .groupby('juiz_str')['vara_str'].nunique()
).fillna(0).astype(int)

N_it

,Cod. Juiz,Qtd. Vara,T1,T2,T3,T4
423,JUIZ_525,40,23,32,29,35
91,JUIZ_1502,31,8,10,13,10
24,JUIZ_1315,31,7,8,13,18
395,JUIZ_486,28,3,8,16,16
88,JUIZ_15,28,6,2,22,9
...,...,...,...,...,...,...
220,JUIZ_202,1,1,1,1,1
221,JUIZ_204,1,1,1,1,1
335,JUIZ_376,1,1,1,1,1
224,JUIZ_208,1,0,1,0,0


## Preparando Dados para Modelo

In [73]:
# Definimos qual trimestre será usado para a otimização
PERIODO_REFERENCIA = 'T4' 

print(f"Período de referência para otimização: {PERIODO_REFERENCIA}")

Período de referência para otimização: T4


In [74]:
# Extraímos os conjuntos únicos que serão usados no modelo

# Conjunto de Juízes (I)
juizes = P_it['juiz_str'].tolist()

# Conjunto de Varas (J)
varas = C_jt['vara_str'].tolist()

# Conjunto de Naturezas/Competências (K)
competencia = E_ik['vm_competencia'].unique().tolist()

print(f"Número de juízes: {len(juizes)}")
print(f"Número de varas: {len(varas)}")
print(f"Número de competência: {len(competencia)}")

Número de juízes: 561
Número de varas: 795
Número de competência: 98


In [75]:
# CONVERSÃO DOS DATAFRAMES PARA DICIONÁRIOS PYOMO
# O Pyomo trabalha melhor com dicionários para parâmetros indexados

# Produtividade dos Juízes (P_it)
produtividade_juizes = dict(zip(P_it['juiz_str'], P_it[PERIODO_REFERENCIA].values.tolist()))

# Especialização dos Juízes (E_ik)
especializacao_juizes = dict(zip(zip(E_ik['juiz_str'], E_ik['vm_competencia']), E_ik['flag']))

# Capacidade das Varas (C_jt)
capacidade_varas = dict(zip(C_jt['vara_str'], C_jt[PERIODO_REFERENCIA].values.tolist()))

# Demanda por Vara e Natureza (D_jk)
demanda_varas = dict(zip(zip(D_jkt['vara_str'], D_jkt['vm_competencia']), D_jkt[PERIODO_REFERENCIA].values.tolist()))

# Máximo de Juízes por Vara (M_jt)
max_juizes_vara = dict(zip(M_jt['vara_str'], M_jt[['T1', 'T2', 'T3', 'T4']].max(axis=1)))

In [84]:
print("\n=== VALIDAÇÃO DOS DADOS ===")
print(f"Produtividade de juízes: {len(produtividade_juizes)} registros")
print(f"Especialização de juízes: {len(especializacao_juizes)} registros")
print(f"Capacidade de varas: {len(capacidade_varas)} registros")
print(f"Demanda de varas: {len(demanda_varas)} registros")
print(f"Máximo de juízes por vara: {len(max_juizes_vara)} registros")

# Verificamos se todos os juízes têm produtividade
juizes_sem_produtividade = [j for j in juizes if j not in produtividade_juizes]
if juizes_sem_produtividade:
    print(f"ATENÇÃO: {len(juizes_sem_produtividade)} juízes sem dados de produtividade")

# Verificamos se todas as varas têm capacidade
varas_sem_capacidade = [v for v in varas if v not in capacidade_varas]
if varas_sem_capacidade:
    print(f"ATENÇÃO: {len(varas_sem_capacidade)} varas sem dados de capacidade")


=== VALIDAÇÃO DOS DADOS ===
Produtividade de juízes: 561 registros
Especialização de juízes: 54978 registros
Capacidade de varas: 795 registros
Demanda de varas: 77910 registros
Máximo de juízes por vara: 795 registros


## Modelagem

In [ ]:
# ConcreteModel = modelo onde os dados são fornecidos diretamente na criação
# É um 'container' que vai armazenar todos os componentes do nosso problema
# Conjuntos (Sets), Parâmetros, Variáveis, Restrições, Função Objetivo
modelo = pyo.ConcreteModel()

# DEFINIÇÃO DE CONJUNTOS (SETS)

# Conjunto I: Juízes
modelo.I = pyo.Set(initialize=juizes)
print(f"✓ Conjunto I (Juízes) criado: {len(modelo.I)} elementos")

# Conjunto J: Varas
modelo.J = pyo.Set(initialize=varas)
print(f"✓ Conjunto J (Varas) criado: {len(modelo.J)} elementos")

# Conjunto K: Competências
modelo.K = pyo.Set(initialize=competencia)
print(f"✓ Conjunto K (Competências) criado: {len(modelo.K)} elementos")

✓ Conjunto I (Juízes) criado: 561 elementos
✓ Conjunto J (Varas) criado: 795 elementos
✓ Conjunto K (Competências) criado: 98 elementos


In [83]:
print("VERIFICAÇÃO DOS CONJUNTOS:")
print(f"   ✓ Conjunto I tem {len(modelo.I)} juízes")
print(f"   ✓ Conjunto J tem {len(modelo.J)} varas") 
print(f"   ✓ Conjunto K tem {len(modelo.K)} naturezas")
print(f"   ✓ Total de combinações (i,j): {len(modelo.I) * len(modelo.J)}")
print(f"   ✓ Total de combinações (j,k): {len(modelo.J) * len(modelo.K)}")

VERIFICAÇÃO DOS CONJUNTOS:
   ✓ Conjunto I tem 561 juízes
   ✓ Conjunto J tem 795 varas
   ✓ Conjunto K tem 98 naturezas
   ✓ Total de combinações (i,j): 445995
   ✓ Total de combinações (j,k): 77910


In [85]:
# DEFINIÇÃO DOS PARÂMETROS (PARAMETERS)

# Parâmetro P[i]: Produtividade ponderada do juiz i
# Na matemática: P_i ∈ ℝ⁺ para todo i ∈ I
# No Pyomo: modelo.P indexado pelo conjunto I
modelo.P = pyo.Param(modelo.I, initialize=produtividade_juizes)
print(f"✓ Parâmetro P[i] (Produtividade dos Juízes) criado")

# Parâmetro C[j]: Capacidade máxima da vara j
# Na matemática: C_j ∈ ℝ⁺ para todo j ∈ J
# No Pyomo: modelo.C indexado pelo conjunto J
modelo.C = pyo.Param(modelo.J, initialize=capacidade_varas)
print(f"✓ Parâmetro C[j] (Capacidade das Varas) criado")

# Parâmetro D[j,k]: Demanda da vara j para natureza k
# Na matemática: D_{j,k} ∈ ℝ⁺ para todo j ∈ J, k ∈ K
# No Pyomo: modelo.D indexado pelos conjuntos J e K
# Usamos default=0 para combinações que não existem nos dados
modelo.D = pyo.Param(modelo.J, modelo.K, initialize=demanda_varas, default=0)
print(f"✓ Parâmetro D[j,k] (Demanda por Vara e Natureza) criado")

# Parâmetro E[i,k]: Especialização do juiz i na natureza k
# Na matemática: E_{i,k} ∈ {0,1} para todo i ∈ I, k ∈ K
# No Pyomo: modelo.E indexado pelos conjuntos I e K
# Usamos default=0 para combinações que não existem nos dados
modelo.E = pyo.Param(modelo.I, modelo.K, initialize=especializacao_juizes, default=0)
print(f"✓ Parâmetro E[i,k] (Especialização dos Juízes) criado")

# Parâmetro M[j]: Máximo de juízes que podem ser alocados à vara j
# Na matemática: M_j ∈ ℕ para todo j ∈ J
# No Pyomo: modelo.M indexado pelo conjunto J
modelo.M = pyo.Param(modelo.J, initialize=max_juizes_vara)
print(f"✓ Parâmetro M[j] (Máximo de Juízes por Vara) criado")

✓ Parâmetro P[i] (Produtividade dos Juízes) criado
✓ Parâmetro C[j] (Capacidade das Varas) criado
✓ Parâmetro D[j,k] (Demanda por Vara e Natureza) criado
✓ Parâmetro E[i,k] (Especialização dos Juízes) criado
✓ Parâmetro M[j] (Máximo de Juízes por Vara) criado


In [86]:
print("VERIFICAÇÃO DOS PARÂMETROS:")
print(f"   ✓ P[i]: {len(modelo.P)} valores de produtividade")
print(f"   ✓ C[j]: {len(modelo.C)} valores de capacidade")
print(f"   ✓ D[j,k]: {len(modelo.J) * len(modelo.K)} combinações possíveis")
print(f"   ✓ E[i,k]: {len(modelo.I) * len(modelo.K)} combinações possíveis")
print(f"   ✓ M[j]: {len(modelo.M)} valores de máximo")

VERIFICAÇÃO DOS PARÂMETROS:
   ✓ P[i]: 561 valores de produtividade
   ✓ C[j]: 795 valores de capacidade
   ✓ D[j,k]: 77910 combinações possíveis
   ✓ E[i,k]: 54978 combinações possíveis
   ✓ M[j]: 795 valores de máximo


In [87]:
# DEFINIÇÃO DAS VARIÁVEIS DE DECISÃO (VARIABLES)

# Variável x[i,j]: Fração da capacidade produtiva do juiz i alocada à vara j
# Na matemática: x_{i,j} ∈ [0,1] para todo i ∈ I, j ∈ J
# No Pyomo: modelo.x indexado pelos conjuntos I e J
modelo.x = pyo.Var(modelo.I, modelo.J, domain=pyo.NonNegativeReals, bounds=(0, 1))
print(f"✓ Variável x[i,j] (Alocação de Juízes às Varas) criada")
print(f"   - Tipo: Contínua")
print(f"   - Domínio: [0, 1] (fração da capacidade)")
print(f"   - Total de variáveis: {len(modelo.I) * len(modelo.J)}")

✓ Variável x[i,j] (Alocação de Juízes às Varas) criada
   - Tipo: Contínua
   - Domínio: [0, 1] (fração da capacidade)
   - Total de variáveis: 445995


In [ ]:
# DEFINIÇÃO DA FUNÇÃO OBJETIVO (OBJECTIVE)

# Função Objetivo: Maximizar a produtividade ponderada total
# Na matemática: max Σᵢ Σⱼ P_i × x_{i,j}
# No Pyomo: Objective com sense=maximize
modelo.obj = pyo.Objective(
    expr=sum(modelo.P[i] * modelo.x[i, j] for i in modelo.I for j in modelo.J),
    sense=pyo.maximize
)

print(f"✓ Função Objetivo criada")
print(f"   - Tipo: Maximização")
print(f"   - Expressão: Σᵢ Σⱼ P[i] × x[i,j]")
print(f"   - Objetivo: Maximizar produtividade total do tribunal")

✓ Função Objetivo criada
   - Tipo: Maximização
   - Expressão: Σᵢ Σⱼ P[i] × x[i,j]
   - Objetivo: Maximizar produtividade total do tribunal


In [89]:
print("VERIFICAÇÃO DA FUNÇÃO OBJETIVO:")
print(f"   ✓ Função objetivo criada: {type(modelo.obj)}")
print(f"   ✓ Sentido: Maximização")
print(f"   ✓ Expressão: Soma da produtividade ponderada")
print(f"   ✓ Variáveis envolvidas: {len(modelo.I) * len(modelo.J)} variáveis x[i,j]")
print(f"   ✓ Parâmetros envolvidos: {len(modelo.P)} valores de produtividade P[i]")

VERIFICAÇÃO DA FUNÇÃO OBJETIVO:
   ✓ Função objetivo criada: <class 'pyomo.core.base.objective.ScalarObjective'>
   ✓ Sentido: Maximização
   ✓ Expressão: Soma da produtividade ponderada
   ✓ Variáveis envolvidas: 445995 variáveis x[i,j]
   ✓ Parâmetros envolvidos: 561 valores de produtividade P[i]
